In [ ]:
# ----------------------------
# Manipulation des données
# ----------------------------
import pandas as pd
import numpy as np
import csv
import json
import mlflow
import mlflow.sklearn

# ----------------------------
# Requêtes HTTP et API
# ----------------------------
import requests
import time
import openrouteservice
from openrouteservice import exceptions

# ----------------------------
# Machine Learning
# ----------------------------

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression  # <-- ajouté
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# ----------------------------
# Géospatiale
# ----------------------------
import geopandas as gpd

# ----------------------------
# Base de données
# ----------------------------
from sqlalchemy import create_engine

# ----------------------------
# Visualisation
# ----------------------------
import matplotlib.pyplot as plt

# ----------------------------
# Dates et calendriers
# ----------------------------
from datetime import date, datetime
import holidays


In [2]:
pip install holidays

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install xgboost

In [91]:
!pip freeze


a2wsgi==1.10.10
accelerate==1.10.1
aiodns==3.5.0
aiohappyeyeballs==2.6.1
aiohttp==3.12.15
aiohttp-cors==0.8.1
aiosignal==1.4.0
aiosqlite==0.21.0
albucore==0.0.24
albumentations==0.5.2
alembic==1.16.5
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.0.0
apache-airflow==3.0.6
apache-airflow-core==3.0.6
apache-airflow-providers-amazon==9.12.0
apache-airflow-providers-common-compat==1.7.3
apache-airflow-providers-common-io==1.6.2
apache-airflow-providers-common-sql==1.27.5
apache-airflow-providers-ftp==3.6.0
apache-airflow-providers-http==4.6.0
apache-airflow-providers-imap==3.4.0
apache-airflow-providers-smtp==2.2.0
apache-airflow-providers-sqlite==3.5.0
apache-airflow-providers-standard==1.6.0
apache-airflow-task-sdk==1.0.6
apispec==6.3.0
argcomplete==3.1.3
asgiref==3.7.2
asn1crypto==1.5.1
asttokens==3.0.0
async-timeout==5.0.1
attrs==25.3.0
Babel==2.13.1
backoff==1.10.0
backports.strenum==1.3.1
beautifulsoup4==4.13.5
blinker==1.6.3
boto3==1.40.21
botocore==1.40.21
braceexpan

## TEMPERATURE JOURNALIERE

### 1- Récupération par API

In [4]:
url_temp = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/temperature-quotidienne-departementale/exports/csv"
params_temp = {
    "use_labels": "true"
}

resp = requests.get(url_temp, params=params_temp)

if resp.status_code == 200:
    with open("temperature_departementale.csv", "wb") as f:
        f.write(resp.content)
    print("Téléchargement réussi : 'temperature_departementale.csv'")
else:
    print("Erreur", resp.status_code)
    print(resp.text)


Téléchargement réussi : 'temperature_departementale.csv'


### 2- On passe la température a la maille de la France

In [30]:
day_temp = pd.read_csv('temperature_departementale.csv', sep = ';')

# Suppose que day_temp est ton DataFrame, avec les colonnes suivantes :
# 'Date', 'Code INSEE département', 'Département', 'TMin (°C)', 'TMax (°C)', 'TMoy (°C)'

# Convertir la colonne Date en datetime et ajouter une colonne Mois
day_temp['Date'] = pd.to_datetime(day_temp['Date'])
day_temp['Mois'] = day_temp['Date'].dt.to_period('M')  # Ex: '2025-06'

# Groupement par date
temp_france_jour = day_temp.groupby("Date").agg({
    "TMoy (°C)": "mean",        # Moyenne des températures moyennes
    "TMin (°C)": "min",         # Température la plus basse observée
    "TMax (°C)": "max"          # Température la plus élevée observée
}).reset_index()

# Renommage
temp_france_jour.rename(columns={
    "TMoy (°C)": "Temp Moyenne France (°C)",
    "TMin (°C)": "Temp Min Observée (°C)",
    "TMax (°C)": "Temp Max Observée (°C)"
}, inplace=True)

In [31]:
temp_france_jour

,Date,Temp Moyenne France (°C),Temp Min Observée (°C),Temp Max Observée (°C)
0,2018-01-01,8.124167,0.10,18.25
1,2018-01-02,9.468750,-0.50,18.40
2,2018-01-03,10.774688,0.20,21.70
3,2018-01-04,11.315625,0.20,19.80
4,2018-01-05,10.228854,1.90,19.83
...,...,...,...,...
2795,2025-08-27,21.418750,10.00,33.43
2796,2025-08-28,19.769063,11.30,33.72
2797,2025-08-29,17.645521,6.90,29.77
2798,2025-08-30,18.510104,7.90,28.13


## CONSOMMATION ELECTRICITE

Vous y trouverez au pas quart d'heure :

Les prévisions de consommation établies la veille (J-1) et celles réactualisées le jour même (J).
Vous y trouverez au pas demi-heure :

La consommation réalisée.
La production selon les différentes filières composant le mix énergétique.
La consommation des pompes dans les Stations de Transfert d'Energie par Pompage (STEP).
Les échanges physiques aux frontières.
Une estimation des émissions de carbone générées par la production d'électricité en France.
Les échanges commerciaux aux frontières.
Le découpage des filières par technologie du mix de production (débute en 2013).

### 1- Récupération par API

In [7]:
# URL de l'API pour les données nationales de consommation électrique (données définitives)
url_conso = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-national-cons-def/exports/json"

# Paramètres de requête
params_conso = {
    "timezone": "Europe/Paris"
}

# Appel API
response = requests.get(url_conso, params=params_conso)

# Traitement de la réponse
if response.status_code == 200:
    records = response.json()
    energie_nationale = pd.json_normalize(records)
    with open("éCO2mix_nationales.json", "wb") as f:
        f.write(response.content)
    print("Téléchargement réussi : 'éCO2mix_nationales.csv'")
    print(energie_nationale.shape)
    print(energie_nationale.head())
else:
    print(f"Erreur API : {response.status_code} – {response.text}")


Téléchargement réussi : 'éCO2mix_nationales.csv'
(455904, 37)
  perimetre               nature        date  heure  \
0    France  Données définitives  2015-01-01  00:30   
1    France  Données définitives  2015-01-01  00:45   
2    France  Données définitives  2015-01-01  01:30   
3    France  Données définitives  2015-01-01  03:00   
4    France  Données définitives  2015-01-01  03:30   

                  date_heure  consommation  prevision_j1  prevision_j  fioul  \
0  2015-01-01T00:30:00+01:00       73481.0         71600        72400  319.0   
1  2015-01-01T00:45:00+01:00           NaN         70750        71500    NaN   
2  2015-01-01T01:30:00+01:00       71505.0         70600        71300  320.0   
3  2015-01-01T03:00:00+01:00       68840.0         67200        67800  320.0   
4  2015-01-01T03:30:00+01:00       67315.0         65400        66400  320.0   

   charbon  ...  gaz_tac  gaz_cogen  gaz_ccg  gaz_autres  \
0   2087.0  ...      9.0       1926    670.0        86.0   
1     

In [8]:
pd.read_json('éCO2mix_nationales.json', orient="records")

,perimetre,nature,date,heure,date_heure,consommation,prevision_j1,prevision_j,fioul,charbon,...,gaz_tac,gaz_cogen,gaz_ccg,gaz_autres,hydraulique_fil_eau_eclusee,hydraulique_lacs,hydraulique_step_turbinage,bioenergies_dechets,bioenergies_biomasse,bioenergies_biogaz
0,France,Données définitives,2015-01-01,00:30,2015-01-01T00:30:00+01:00,73481.0,71600,72400,319.0,2087.0,...,9.0,1926,670.0,86.0,4616.0,3561.0,1487.0,507.0,180.0,210.0
1,France,Données définitives,2015-01-01,00:45,2015-01-01T00:45:00+01:00,NaN,70750,71500,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,France,Données définitives,2015-01-01,01:30,2015-01-01T01:30:00+01:00,71505.0,70600,71300,320.0,1589.0,...,10.0,1924,420.0,86.0,4369.0,2623.0,1301.0,512.0,180.0,212.0
3,France,Données définitives,2015-01-01,03:00,2015-01-01T03:00:00+01:00,68840.0,67200,67800,320.0,1288.0,...,9.0,1926,238.0,86.0,4143.0,1807.0,629.0,513.0,182.0,213.0
4,France,Données définitives,2015-01-01,03:30,2015-01-01T03:30:00+01:00,67315.0,65400,66400,320.0,1251.0,...,9.0,1924,243.0,86.0,4026.0,1768.0,267.0,518.0,183.0,212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455899,France,Données définitives,2020-07-27,00:15,2020-07-27T00:15:00+02:00,NaN,41000,41650,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455900,France,Données définitives,2020-07-27,00:30,2020-07-27T00:30:00+02:00,40641.0,39800,40500,861.0,-37.0,...,337.0,436,2400.0,53.0,3121.0,2318.0,755.0,524.0,288.0,326.0
455901,France,Données définitives,2020-07-27,02:45,2020-07-27T02:45:00+02:00,NaN,34750,35650,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455902,France,Données définitives,2020-07-27,04:45,2020-07-27T04:45:00+02:00,NaN,33000,34000,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2- On passe a la maille de la journée

In [9]:
def aggregation_journaliere(df):
    df = df.copy()

    # 1. Colonnes à renommer avec unité
    colonnes_MW = [
        'consommation', 'prevision_j1', 'prevision_j', 'fioul', 'charbon', 'gaz',
        'nucleaire', 'eolien', 'solaire', 'hydraulique', 'pompage', 'bioenergies',
        'ech_physiques', 'ech_comm_angleterre', 'ech_comm_espagne',
        'ech_comm_italie', 'ech_comm_suisse', 'ech_comm_allemagne_belgique',
        'fioul_tac', 'fioul_cogen', 'fioul_autres', 'gaz_tac', 'gaz_cogen',
        'gaz_ccg', 'gaz_autres', 'hydraulique_fil_eau_eclusee',
        'hydraulique_lacs', 'hydraulique_step_turbinage', 'bioenergies_dechets',
        'bioenergies_biomasse', 'bioenergies_biogaz'
    ]
    colonnes_CO2 = ['taux_co2']
    colonnes_preserves = ['perimetre', 'nature']

    # 2. Renommage avec unités
    colonnes_renommees = {col: f"{col}_MW" for col in colonnes_MW}
    colonnes_renommees.update({col: f"{col}_gCO2_kWh" for col in colonnes_CO2})
    df.rename(columns=colonnes_renommees, inplace=True)

    # 3. Gestion de la colonne date_heure
    if 'date_heure' in df.columns:
        df['date_heure'] = pd.to_datetime(df['date_heure'], errors='coerce', utc=True)
        df['date_heure'] = df['date_heure'].dt.tz_convert(None)
    else:
        df['heure'] = df['heure'].astype(str).str.zfill(5)
        df['date_heure'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['heure'], errors='coerce')

    df.dropna(subset=['date_heure'], inplace=True)

    # Supprimer timezone si présent
    if isinstance(df['date_heure'].dtype, pd.DatetimeTZDtype):
        df['date_heure'] = df['date_heure'].dt.tz_convert(None)

    # 4. On définit un index temporel
    df.set_index('date_heure', inplace=True)

    # Vérifier que l'index est bien un DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("L'index doit être un DatetimeIndex pour interpoler avec method='time'")

    # 5. Colonnes numériques à interpoler (MW et CO2)
    colonnes_moyenne = list(colonnes_renommees.values())
    for col in colonnes_moyenne:
        if col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')  # assure que ce sont bien des numériques
                df[col] = df[col].interpolate(method='time', limit_direction='both')
            except Exception as e:
                print(f"⚠️ Problème avec la colonne {col} : {e}")

    # 6. Ajouter la colonne 'date' pour grouper
    df['date'] = df.index.date

    # 7. Définir les règles d’agrégation
    aggregation_dict = {col: 'mean' for col in colonnes_moyenne}
    for col in colonnes_preserves:
        if col in df.columns:
            aggregation_dict[col] = 'first'

    df_reset = df.reset_index()

    # Supprime colonne 'date' si présente pour éviter doublons
    if 'date' in df_reset.columns:
        df_reset = df_reset.drop(columns=['date'])

    # Recalcule proprement la colonne date à partir de date_heure
    df_reset['date'] = df_reset['date_heure'].dt.date

    # Agrégation
    energie_nationale_jour = df_reset.groupby('date', as_index=False).agg(aggregation_dict)

    return energie_nationale_jour




In [10]:
energie_nationale_jour = aggregation_journaliere(energie_nationale)

### 3- On crée de nouvelles variables

In [11]:
# Liste des colonnes en MW à convertir
cols_mw = [col for col in energie_nationale_jour.columns if col.endswith('_MW') and col not in ['pompage_MW']]

# Conversion MW moyen ➜ MWh total journalier
for col in cols_mw:
    energie_nationale_jour[col.replace('_MW', '_MWh')] = energie_nationale_jour[col] * 24

In [12]:
energie_nationale_jour

,date,consommation_MW,prevision_j1_MW,prevision_j_MW,fioul_MW,charbon_MW,gaz_MW,nucleaire_MW,eolien_MW,solaire_MW,...,gaz_tac_MWh,gaz_cogen_MWh,gaz_ccg_MWh,gaz_autres_MWh,hydraulique_fil_eau_eclusee_MWh,hydraulique_lacs_MWh,hydraulique_step_turbinage_MWh,bioenergies_dechets_MWh,bioenergies_biomasse_MWh,bioenergies_biogaz_MWh
0,2011-12-31,58054.500000,57325.000000,57162.500000,492.000000,25.000000,3818.250000,52578.250000,3581.500000,0.000000,...,-72.000000,68592.000000,11976.000000,2256.00000,133320.000000,29136.000000,18192.000000,11472.000000,3120.000000,3864.000000
1,2012-01-01,51303.380208,50849.479167,50232.812500,492.687500,15.197917,3788.421875,44546.822917,3931.500000,130.312500,...,-72.000000,68592.000000,11976.000000,2256.00000,133320.000000,29136.000000,18192.000000,11472.000000,3120.000000,3864.000000
2,2012-01-02,61123.401042,60759.375000,60166.666667,491.536458,8.088542,4053.234375,53828.984375,3067.010417,88.895833,...,-72.000000,68592.000000,11976.000000,2256.00000,133320.000000,29136.000000,18192.000000,11472.000000,3120.000000,3864.000000
3,2012-01-03,67774.286458,66172.916667,66610.416667,514.583333,3.479167,4022.161458,58158.281250,4759.635417,141.000000,...,-72.000000,68592.000000,11976.000000,2256.00000,133320.000000,29136.000000,18192.000000,11472.000000,3120.000000,3864.000000
4,2012-01-04,68121.432292,67304.687500,67043.750000,546.020833,0.182292,3714.536458,57653.942708,4330.927083,180.312500,...,-72.000000,68592.000000,11976.000000,2256.00000,133320.000000,29136.000000,18192.000000,11472.000000,3120.000000,3864.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,2024-12-27,62538.359375,59038.020833,61748.437500,93.796875,17.635417,4854.140625,52852.057292,956.302083,1593.036458,...,2583.250000,15525.000000,80203.500000,18187.25000,157792.750000,102078.625000,33956.625000,13275.625000,8866.625000,8087.750000
4746,2024-12-28,62952.927083,60131.250000,62422.395833,301.802083,17.375000,5346.067708,52005.072917,532.645833,1734.770833,...,2937.500000,15195.625000,92064.000000,18109.87500,152522.500000,98949.250000,20583.750000,13019.625000,9033.500000,8116.750000
4747,2024-12-29,61500.364583,61414.062500,61236.458333,92.375000,18.932292,2878.979167,50919.359375,629.401042,1311.270833,...,2356.875000,14858.000000,33929.625000,17953.50000,141680.375000,74105.250000,13689.875000,12941.125000,8765.625000,8012.750000
4748,2024-12-30,65457.796875,66238.541667,65278.125000,259.765625,17.932292,3320.250000,50056.005208,1011.083333,981.895833,...,2579.375000,15931.000000,43196.875000,17974.25000,138607.250000,77296.375000,19521.250000,12587.875000,8292.125000,7971.375000


## CONSOMMATION QUOTIDIENNE CORRIGEE DE LA METEO

### 1- Récuperation par API par batch

In [13]:
import requests

url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-quotidienne-corrige-brute/records"
params = {
    "limit": 20,
    "offset": 0
}
response = requests.get(url, params=params)
print(response.status_code)
print(response.json())


200
{'total_count': 3862, 'results': [{'date': '2024-07-08', 'consommation_a_temperature_normale_mw': 43476}, {'date': '2024-05-11', 'consommation_a_temperature_normale_mw': 37289}, {'date': '2024-04-26', 'consommation_a_temperature_normale_mw': 44657}, {'date': '2024-04-23', 'consommation_a_temperature_normale_mw': 45876}, {'date': '2024-03-10', 'consommation_a_temperature_normale_mw': 51853}, {'date': '2024-02-16', 'consommation_a_temperature_normale_mw': 61578}, {'date': '2024-02-01', 'consommation_a_temperature_normale_mw': 64995}, {'date': '2023-12-31', 'consommation_a_temperature_normale_mw': 60259}, {'date': '2023-12-22', 'consommation_a_temperature_normale_mw': 64883}, {'date': '2023-12-20', 'consommation_a_temperature_normale_mw': 65711}, {'date': '2023-11-22', 'consommation_a_temperature_normale_mw': 60396}, {'date': '2023-11-21', 'consommation_a_temperature_normale_mw': 59850}, {'date': '2023-10-21', 'consommation_a_temperature_normale_mw': 43461}, {'date': '2023-10-20', 'co

In [14]:


def fetch_data_batch(limit=20, offset=0):
    url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-quotidienne-corrige-brute/records"
    params = {
        "limit": limit,
        "offset": offset
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def main():
    limit = 20
    offset = 0
    total_records = None
    csv_filename = "consommation_quotidienne.csv"

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ["date", "consommation_a_temperature_normale_mw"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        while True:
            data = fetch_data_batch(limit=limit, offset=offset)
            results = data.get("results", [])

            if total_records is None:
                total_records = data.get("total_count", 0)
                print(f"Total records to fetch: {total_records}")

            if not results:
                print("Aucun enregistrement récupéré, fin de la récupération.")
                break

            print(f"Batch {offset//limit + 1} : {len(results)} enregistrements")

            for record in results:
                date = record.get("date", "")
                conso = record.get("consommation_a_temperature_normale_mw", "")
                writer.writerow({
                    "date": date,
                    "consommation_a_temperature_normale_mw": conso
                })

            offset += limit
            if offset >= total_records:
                print("Fin de récupération de toutes les données.")
                break

    print(f"Données enregistrées dans le fichier {csv_filename}")

if __name__ == "__main__":
    main()


Total records to fetch: 3862
Batch 1 : 20 enregistrements
Batch 2 : 20 enregistrements
Batch 3 : 20 enregistrements
Batch 4 : 20 enregistrements
Batch 5 : 20 enregistrements
Batch 6 : 20 enregistrements
Batch 7 : 20 enregistrements
Batch 8 : 20 enregistrements
Batch 9 : 20 enregistrements
Batch 10 : 20 enregistrements
Batch 11 : 20 enregistrements
Batch 12 : 20 enregistrements
Batch 13 : 20 enregistrements
Batch 14 : 20 enregistrements
Batch 15 : 20 enregistrements
Batch 16 : 20 enregistrements
Batch 17 : 20 enregistrements
Batch 18 : 20 enregistrements
Batch 19 : 20 enregistrements
Batch 20 : 20 enregistrements
Batch 21 : 20 enregistrements
Batch 22 : 20 enregistrements
Batch 23 : 20 enregistrements
Batch 24 : 20 enregistrements
Batch 25 : 20 enregistrements
Batch 26 : 20 enregistrements
Batch 27 : 20 enregistrements
Batch 28 : 20 enregistrements
Batch 29 : 20 enregistrements
Batch 30 : 20 enregistrements
Batch 31 : 20 enregistrements
Batch 32 : 20 enregistrements
Batch 33 : 20 enregi

In [15]:
conso_corrigee = pd.read_csv('consommation_quotidienne.csv')

## JOUR FERIES ET VACANCES SCOLAIRE

### 1- Constitution du dataframe

In [16]:
years = range(2012, 2025)  # 2025 exclusif, donc jusqu'à 2024 inclus
fr_holidays = holidays.France(years=years)
df_holidays = pd.DataFrame(list(fr_holidays.items()), columns=['date', 'holiday'])


## ON MERGE LES DATASETS

In [74]:
# Assure-toi que les deux colonnes 'date' sont bien au bon format (datetime.date ou datetime64[ns])
energie_nationale_jour['date'] = pd.to_datetime(energie_nationale_jour['date']).dt.date
conso_corrigee['date'] = pd.to_datetime(conso_corrigee['date']).dt.date
temp_france_jour = temp_france_jour.rename(columns = {"Date" : 'date'})
temp_france_jour['date'] = pd.to_datetime(conso_corrigee['date']).dt.date

# Réaliser une jointure externe gauche (ou autre type selon ton besoin)
# Ici : on garde toutes les lignes de energie_nationale et ajoute les colonnes de conso_corrigee si match
df_merge = energie_nationale_jour.merge(conso_corrigee, on='date', how='left').merge(df_holidays, on='date', how='left').merge(temp_france_jour, on='date', how='left')

In [75]:
df_merge[df_merge['date'] == pd.to_datetime('2024-07-08').date()]

,date,consommation_MW,prevision_j1_MW,prevision_j_MW,fioul_MW,charbon_MW,gaz_MW,nucleaire_MW,eolien_MW,solaire_MW,...,hydraulique_lacs_MWh,hydraulique_step_turbinage_MWh,bioenergies_dechets_MWh,bioenergies_biomasse_MWh,bioenergies_biogaz_MWh,consommation_a_temperature_normale_mw,holiday,Temp Moyenne France (°C),Temp Min Observée (°C),Temp Max Observée (°C)
4573,2024-07-08,43846.270833,42977.604167,42918.75,120.234375,17.973958,656.682292,38784.375,1656.817708,5352.885417,...,53794.125,22340.0,13052.875,8343.5,8017.0,43476.0,NaN,8.124167,0.1,18.25


#### la colonne "consommation_a_temperature_normale_mw" est a la maille de la semaine, on remplit les NaN avec une interpolation temporelle pour la colonne 


In [76]:
df_merge["date"] = pd.to_datetime(df_merge["date"])  # Assure-toi que c’est bien un datetime
df_merge.set_index("date", inplace=True)

# Interpolation maintenant possible
df_merge["consommation_a_temperature_normale_mw"] = (
    df_merge["consommation_a_temperature_normale_mw"]
    .interpolate(method="time")
    .fillna(method="bfill")  # au cas où au début il manque des valeurs
)

# Tu peux ensuite réinitialiser l'index si nécessaire
df_merge.reset_index(inplace=True)
df_merge["date"] = pd.to_datetime(df_merge["date"]) 


C:\Users\Admin\AppData\Local\Temp\ipykernel_13912\3790880133.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merge["consommation_a_temperature_normale_mw"]


In [77]:
df_merge.columns

Index(['date', 'consommation_MW', 'prevision_j1_MW', 'prevision_j_MW',
       'fioul_MW', 'charbon_MW', 'gaz_MW', 'nucleaire_MW', 'eolien_MW',
       'solaire_MW', 'hydraulique_MW', 'pompage_MW', 'bioenergies_MW',
       'ech_physiques_MW', 'ech_comm_angleterre_MW', 'ech_comm_espagne_MW',
       'ech_comm_italie_MW', 'ech_comm_suisse_MW',
       'ech_comm_allemagne_belgique_MW', 'fioul_tac_MW', 'fioul_cogen_MW',
       'fioul_autres_MW', 'gaz_tac_MW', 'gaz_cogen_MW', 'gaz_ccg_MW',
       'gaz_autres_MW', 'hydraulique_fil_eau_eclusee_MW',
       'hydraulique_lacs_MW', 'hydraulique_step_turbinage_MW',
       'bioenergies_dechets_MW', 'bioenergies_biomasse_MW',
       'bioenergies_biogaz_MW', 'taux_co2_gCO2_kWh', 'perimetre', 'nature',
       'consommation_MWh', 'prevision_j1_MWh', 'prevision_j_MWh', 'fioul_MWh',
       'charbon_MWh', 'gaz_MWh', 'nucleaire_MWh', 'eolien_MWh', 'solaire_MWh',
       'hydraulique_MWh', 'bioenergies_MWh', 'ech_physiques_MWh',
       'ech_comm_angleterre_MWh',

#### Les températures ne sont qu'a partir de 2018 on fill les nan avec la valeur moyenne pour les meme jours aux années d'avant et si il reste des na on fait avec la moyenne au mois


In [78]:
# Assurer que la colonne date est bien en datetime
df_merge['date'] = pd.to_datetime(df_merge['date'])

# Colonnes de température à imputer
temp_cols = ["Temp Moyenne France (°C)", "Temp Min Observée (°C)", "Temp Max Observée (°C)"]

# Extraire jour et mois depuis 'date'
df_merge['day'] = df_merge['date'].dt.day
df_merge['month'] = df_merge['date'].dt.month

# Créer une colonne 'day_month' pour grouper par jour et mois
df_merge['day_month'] = df_merge['day'].astype(str) + "-" + df_merge['month'].astype(str)

# 1️⃣ Remplissage par moyenne du même jour (toutes années confondues)
for col in temp_cols:
    df_merge[col] = df_merge.groupby('day_month')[col].transform(lambda x: x.fillna(x.mean()))

# 2️⃣ Remplissage des NaN restants par moyenne du mois
for col in temp_cols:
    df_merge[col] = df_merge.groupby('month')[col].transform(lambda x: x.fillna(x.mean()))

# Supprimer les colonnes auxiliaires
df_merge.drop(columns=['day', 'month', 'day_month'], inplace=True)



In [80]:
df_merge[df_merge['date'] == '2024-07-08']

,date,consommation_MW,prevision_j1_MW,prevision_j_MW,fioul_MW,charbon_MW,gaz_MW,nucleaire_MW,eolien_MW,solaire_MW,...,hydraulique_lacs_MWh,hydraulique_step_turbinage_MWh,bioenergies_dechets_MWh,bioenergies_biomasse_MWh,bioenergies_biogaz_MWh,consommation_a_temperature_normale_mw,holiday,Temp Moyenne France (°C),Temp Min Observée (°C),Temp Max Observée (°C)
4573,2024-07-08,43846.270833,42977.604167,42918.75,120.234375,17.973958,656.682292,38784.375,1656.817708,5352.885417,...,53794.125,22340.0,13052.875,8343.5,8017.0,43476.0,NaN,8.124167,0.1,18.25


## PREPROCESSING

In [81]:
# 1. Copie du DataFrame
df_ml = df_merge.copy()

# 2. Standardisation de la colonne 'date' en datetime.date
df_ml['date'] = pd.to_datetime(df_ml['date']).dt.date

# 3. Génération de features temporelles utiles
df_ml['year'] = pd.to_datetime(df_ml['date']).dt.year
df_ml['month'] = pd.to_datetime(df_ml['date']).dt.month
df_ml['day'] = pd.to_datetime(df_ml['date']).dt.day
df_ml['dayofweek'] = pd.to_datetime(df_ml['date']).dt.dayofweek

def est_weekend(date_str):
    date = datetime.strptime(date_str, "%Y-%m-%d")
    # 5 = samedi, 6 = dimanche en Python
    return date.weekday() >= 5
df_ml['is_weekend'] = df_ml['date'].astype(str).apply(est_weekend)


# 4. Création de l’indicateur binaire 'is_holiday'
# fr_holidays est supposé être un dict ou un set de dates fériées
df_ml['is_holiday'] = df_ml['date'].apply(lambda d: 1 if d in fr_holidays else 0)

# (Optionnel) Pour garder aussi le nom du jour férié
df_ml['holiday_name'] = df_ml['date'].apply(lambda d: fr_holidays.get(d) if d in fr_holidays else "")

# 5. Conversion de 'is_weekend' en entier binaire (en supposant que c’est bool ou int déjà)
df_ml['is_weekend'] = df_ml['is_weekend'].astype(int)

# 6. Remplacement des NaN dans les colonnes énergétiques (en MW et MWh) par 0
energy_cols = [col for col in df_ml.columns if col.endswith('_MW') or col.endswith('_MWh')]
df_ml[energy_cols] = df_ml[energy_cols].fillna(0)

# 7. Création de la variable cible : consommation du jour suivant (puissance moyenne MW)
df_ml['target_conso_j+1'] = df_ml['consommation_MW'].shift(-1)

# 8. Suppression des lignes où la variable cible est manquante (la dernière ligne normalement)
df_ml = df_ml.dropna(subset=['target_conso_j+1'])

# 9. Réorganisation des colonnes : la cible en dernière colonne
cols_order = [c for c in df_ml.columns if c != 'target_conso_j+1'] + ['target_conso_j+1']
df_ml = df_ml[cols_order]

# (Optionnel) Afficher les colonnes finales
print(df_ml.columns)


Index(['date', 'consommation_MW', 'prevision_j1_MW', 'prevision_j_MW',
       'fioul_MW', 'charbon_MW', 'gaz_MW', 'nucleaire_MW', 'eolien_MW',
       'solaire_MW', 'hydraulique_MW', 'pompage_MW', 'bioenergies_MW',
       'ech_physiques_MW', 'ech_comm_angleterre_MW', 'ech_comm_espagne_MW',
       'ech_comm_italie_MW', 'ech_comm_suisse_MW',
       'ech_comm_allemagne_belgique_MW', 'fioul_tac_MW', 'fioul_cogen_MW',
       'fioul_autres_MW', 'gaz_tac_MW', 'gaz_cogen_MW', 'gaz_ccg_MW',
       'gaz_autres_MW', 'hydraulique_fil_eau_eclusee_MW',
       'hydraulique_lacs_MW', 'hydraulique_step_turbinage_MW',
       'bioenergies_dechets_MW', 'bioenergies_biomasse_MW',
       'bioenergies_biogaz_MW', 'taux_co2_gCO2_kWh', 'perimetre', 'nature',
       'consommation_MWh', 'prevision_j1_MWh', 'prevision_j_MWh', 'fioul_MWh',
       'charbon_MWh', 'gaz_MWh', 'nucleaire_MWh', 'eolien_MWh', 'solaire_MWh',
       'hydraulique_MWh', 'bioenergies_MWh', 'ech_physiques_MWh',
       'ech_comm_angleterre_MWh',

In [82]:
X = df_ml.drop(['target_conso_j+1', 'holiday_name', 'perimetre', 'nature', 'holiday', 'date'], axis=1)

In [83]:
X

,consommation_MW,prevision_j1_MW,prevision_j_MW,fioul_MW,charbon_MW,gaz_MW,nucleaire_MW,eolien_MW,solaire_MW,hydraulique_MW,...,consommation_a_temperature_normale_mw,Temp Moyenne France (°C),Temp Min Observée (°C),Temp Max Observée (°C),year,month,day,dayofweek,is_weekend,is_holiday
0,58054.500000,57325.000000,57162.500000,492.000000,25.000000,3818.250000,52578.250000,3581.500000,0.000000,7881.500000,...,60955.0,14.381493,4.133333,25.906667,2011,12,31,5,1,0
1,51303.380208,50849.479167,50232.812500,492.687500,15.197917,3788.421875,44546.822917,3931.500000,130.312500,7627.692708,...,60955.0,12.866533,2.500000,24.917143,2012,1,1,6,1,1
2,61123.401042,60759.375000,60166.666667,491.536458,8.088542,4053.234375,53828.984375,3067.010417,88.895833,8621.890625,...,60955.0,15.339125,3.760000,26.848000,2012,1,2,0,0,0
3,67774.286458,66172.916667,66610.416667,514.583333,3.479167,4022.161458,58158.281250,4759.635417,141.000000,10093.994792,...,60955.0,14.340336,3.155556,25.581111,2012,1,3,1,0,0
4,68121.432292,67304.687500,67043.750000,546.020833,0.182292,3714.536458,57653.942708,4330.927083,180.312500,10227.218750,...,60955.0,13.283611,3.044444,24.346667,2012,1,4,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4744,57593.015625,56476.041667,56426.562500,94.046875,18.348958,2523.890625,51791.619792,2390.062500,1526.296875,10437.135417,...,37881.0,14.258125,3.185714,25.557143,2024,12,26,3,0,0
4745,62538.359375,59038.020833,61748.437500,93.796875,17.635417,4854.140625,52852.057292,956.302083,1593.036458,12242.890625,...,37881.0,12.394434,2.108750,23.752500,2024,12,27,4,0,0
4746,62952.927083,60131.250000,62422.395833,301.802083,17.375000,5346.067708,52005.072917,532.645833,1734.770833,11335.630208,...,37881.0,12.041934,0.650000,24.143750,2024,12,28,5,1,0
4747,61500.364583,61414.062500,61236.458333,92.375000,18.932292,2878.979167,50919.359375,629.401042,1311.270833,9561.635417,...,37881.0,12.488190,2.273750,23.918750,2024,12,29,6,1,0


## MODELES

In [84]:
# Séparation des features et de la target
X = df_ml.drop(['target_conso_j+1', 'holiday_name', 'perimetre', 'nature', 'holiday', 'date'], axis=1)
y = df_ml['target_conso_j+1']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation pour le modèle linéaire
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [89]:
# ------------------------------
# Initialiser les modèles
# ------------------------------
models = {
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, n_estimators=100)
}

# ------------------------------
# Stocker les scores
# ------------------------------
results = {}

for name, model in models.items():
    if name == "LinearRegression":
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {"model": model, "RMSE": rmse, "R2": r2}
    
    print(f"{name} : RMSE={rmse:.2f}, R²={r2:.3f}")

LinearRegression : RMSE=2883.49, R²=0.926
DecisionTree : RMSE=2537.34, R²=0.942
XGBoost : RMSE=1802.96, R²=0.971


### le meilleur modele doit etre envoyé sur MLFLOW

In [90]:
best_model_name = max(results, key=lambda k: results[k]["R2"])
best_model = results[best_model_name]["model"]

print(f"\n✅ Meilleur modèle : {best_model_name}")



✅ Meilleur modèle : XGBoost


In [ ]:
mlflow.set_experiment("Electricity_Forecasting")

with mlflow.start_run(run_name=best_model_name):
    mlflow.log_params({"model_name": best_model_name})
    mlflow.log_metrics({
        "RMSE": results[best_model_name]["RMSE"],
        "R2": results[best_model_name]["R2"]
    })
    mlflow.sklearn.log_model(best_model, artifact_path="model")